# HD Calculations

### Imports

# HD hull

In [96]:
import pandas as pd
import plotly.express as px
import os
import sqlite3
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import numpy as np

## Functions

In [367]:
def calcArea(x, y, in_hd):
    test_nan = np.sum(np.isnan(x))
    # print(test_nan)
    valid_values = len(x)-test_nan
    x = x[0:valid_values]
    y = y[0:valid_values]
    in_hd = (in_hd/10) * valid_values
    
    m = 35
    n = 35
    x_min = np.array([-m, m, -m, m, -m, m, -m, m, -m, m])
    y_min = np.array([-n, -n, n, n, -n, -n, n, n, -n, -n])
    hull_min = ConvexHull(np.column_stack((x_min, y_min)))
    a = 214
    b = 107
    x_max = np.array([-a, a, -a, a, -a, a, -a, a, -a, a])
    y_max = np.array([-b, -b, b, b, -b, -b, b, b, -b, -b])
    hull_max = ConvexHull(np.column_stack((x_max, y_max)))

    hd_max = in_hd
    slices = valid_values+1
    hd_range = np.linspace(0, hd_max, slices)
    # hd_range = np.append(hd_range, 5)
    area_range = np.linspace(hull_min.volume, hull_max.volume, 10)
    # print(hd_range)
    # print(area_range)
    if test_nan < 8:
        in_area = ConvexHull(np.hstack((x,y))).volume
        below_range = np.sum(in_area >= area_range)
        # print(np.sum(in_area > area_range))
        return(len(x)-hd_range[below_range])
    else:
        return(len(x))


def calcDistance(x, y, in_hd):
    test_nan = np.sum(np.isnan(x))
    # print(test_nan)
    valid_values = len(x)-test_nan
    x = x[0:valid_values]
    y = y[0:valid_values]
    in_hd = (in_hd/10) * valid_values

    in_distance = np.sum(np.sqrt(x**2+y**2))
    m = 35
    n = 35
    x_min = np.array([-m, m, -m, m, 0, m, 0, m, 0, 0])
    y_min = np.array([-n, -n, 0, 0, -n, -n, n, n, 0, 0])
    optimum_distance = np.sum(np.sqrt(x_min**2+y_min**2)) 

    a = 214
    b = 107
    x_max = np.array([-a, a, -a, a, -107.5, 107.5, -107.5, 107.5, -a, a])
    y_max = np.array([-b, -b, b, b, -b, -b, b, b, 0, 0])
    worst_distance = np.sum(np.sqrt(x_max**2+y_max**2))

    hd_max = in_hd
    slices = valid_values+1
    hd_range = np.linspace(0, hd_max, slices)
    # hd_range = np.append(hd_range, 5)
    area_range = np.linspace(optimum_distance, worst_distance, 10)
    # print(hd_range)
    # print(area_range)

    below_range = np.sum(in_distance >= area_range)
    # print(np.sum(in_area > area_range))
    return(len(x)-hd_range[below_range])

def calcError(x, y, in_hd):
    test_nan = np.sum(np.isnan(x))
    # print(test_nan)
    valid_values = len(x)-test_nan
    x = x[0:valid_values]
    y = y[0:valid_values]
    in_hd = (in_hd/10) * valid_values

    distance = np.sqrt(x**2+2*y**2)
    mean_distance = np.mean(distance)
    ce = np.sum(distance)/len(x)
    ve = np.sqrt(np.sum((distance - mean_distance)**2/len(x)))
    current_error = ce + ve
    # print(current_error)
    m = 35
    n = 35
    x_min = np.array([-m, m, -m, m, 0, m, 0, m, 0, 0])
    y_min = np.array([-n, -n, 0, 0, -n, -n, n, n, 0, 0])

    distance = np.sqrt(x_min**2+2*y_min**2)
    mean_distance = np.mean(distance)
    ce = np.sum(distance)/len(x_min)
    ve = np.sqrt(np.sum((distance - mean_distance)**2/len(x_min)))
    optimum_error = ce + ve

    a = 214
    b = 107
    x_max = np.array([-a, a, -a, a, -107.5, 107.5, -107.5, 107.5, -a, a])
    y_max = np.array([-b, -b, b, b, -b, -b, b, b, 0, 0])

    distance = np.sqrt(x_max**2+2*y_max**2)
    mean_distance = np.mean(distance)
    ce = np.sum(distance)/len(x_max)
    ve = np.sqrt(np.sum((distance - mean_distance)**2/len(x_max)))
    worst_error = ce + ve

    hd_max = in_hd
    slices = int(round(in_hd/0.05) + 1)
    hd_range = np.linspace(0, hd_max, slices)
    # hd_range = np.append(hd_range, 5)
    area_range = np.linspace(optimum_error, worst_error, slices-1)
    # print(hd_range)
    # print(area_range)

    below_range = np.sum(current_error >= area_range)
    # print(np.sum(in_area > area_range))
    return(len(x)-hd_range[below_range])

## SQL Database connection    

In [318]:
connection = sqlite3.connect("trampoline.db")
cursor = connection.cursor()

In [319]:
sql_str = "SELECT * from 'ed63e921b45f7b0137a59d2958912b22'"
df_exercisedata = pd.read_sql(sql_str, connection)
df_exercisedata['x'] = df_exercisedata.apply(lambda x: int(x['x']), axis=1)
x = df_exercisedata[['x']].values
df_exercisedata['y'] = df_exercisedata.apply(lambda x: int(x['y']), axis=1)
y = df_exercisedata[['y']].values
hull = ConvexHull(df_exercisedata[['x','y']])
hd = calcArea(hull.volume, in_hd=3)
print(hd)

plt.scatter(x, y)
plt.plot(x[hull.vertices], y[hull.vertices], 'r--', lw=2)

plt.plot(x[hull.vertices[0]], y[hull.vertices[0]], 'ro', alpha=0.5)

m = 35
n = 35
x_min = np.array([-m, m, -m, m, -m, m, -m, m, -m, m])
y_min = np.array([-n, -n, n, n, -n, -n, n, n, -n, -n])
hull_min = ConvexHull(np.column_stack((x_min, y_min)))
print('Volume exercise: {0}'.format(hull.volume))
print('Volume min: {0}'.format(hull_min.volume))
plt.scatter(x_min, y_min)
plt.plot(x_min[hull_min.vertices], y_min[hull_min.vertices], 'r--', lw=2)
x_max = np.array([-214, 214, -214, 214, -214, 214, -214, 214, -214, 214])
y_max = np.array([-107, -107, 107, 107, -107, -107, 107, 107, -107, -107])
hull_max = ConvexHull(np.column_stack((x_max, y_max)))
print('Volume max: {0}'.format(hull_max.volume))
plt.scatter(x_max, y_max)
plt.plot(x_max[hull_max.vertices], y_max[hull_max.vertices], 'r--', lw=2)

TypeError: calcArea() missing 1 required positional argument: 'y'

# HD Distance

In [240]:
m = 35
n = 35
x_min = np.array([-m, m, -m, m, 0, m, 0, m, 0, 0])
y_min = np.array([-n, -n, 0, 0, -n, -n, n, n, 0, 0])
optimum_distance = np.sum(np.sqrt(x_min**2+y_min**2)) 

a = 214
b = 107
x_max = np.array([-a, a, -a, a, -107.5, 107.5, -107.5, 107.5, -a, a])
y_max = np.array([-b, -b, b, b, -b, -b, b, b, 0, 0])
worst_distance = np.sum(np.sqrt(x_max**2+y_max**2)) 

calcDistance(np.sum(np.sqrt(df_exercisedata['x']**2+df_exercisedata['y']**2)), 5)

1.0

# HD Error

In [228]:
distance = np.sqrt(x**2+2*y**2)
mean_distance = np.mean(distance)
ce = np.sum(distance)/len(x)
ve = np.sqrt(np.sum((distance - mean_distance)**2/len(x)))

In [247]:
calcError(x_max, y_max, 3)

256.3135837568132


3.0

# HD rank analysis

In [390]:
event_phases = ['Qualification', 'Semi', 'Final']
event_gender = ['Women', 'Men']
df = pd.read_sql("SELECT * from ranklists", connection)
df['Event Name'] = df["Year"].astype(str) + " " + df["Event"]
events = df["Event Name"].unique().tolist()
event_str = events[0]
df_event = df[((df['Event Name']==event_str) & (df['Phase']=='Qualification') & (df['Gender']=='Women'))].copy()

### 2022 EUCH Qualification

In [391]:
hd_distance3_list = []
hd_distance5_list = []
hd_error3_list = []
hd_error5_list = []
hd_hull3_list = []
hd_hull5_list = []
for idx, row in df_event.iterrows():
    gender = row['Gender']
    athlete = row['Name']
    athlete_name = athlete.split()
    athlete_name = athlete_name[0].capitalize() + ' ' + athlete_name[1]
    if row['Routine'] == '1st':
        exercise = 0
    elif row['Routine'] == '2nd':
        exercise = 1
    phase_select = df_event["Phase"].loc[int(exercise)]
    routine_select = df_event["Routine"].loc[int(exercise)]
    sql_str = "SELECT * from '" + event_str.replace(" ", "_").lower() + "_" + gender.lower() + "' where name=" + "'" + athlete_name + "' and phase=" + "'" + phase_select + "' and Routine=" + "'" + routine_select + "'"
    df_athlete = pd.read_sql(sql_str, connection)
    if len(df_athlete) > 0:
        hash_val = df_athlete["Hash"].iloc[0]

        if hash_val is not '':
            sql_str = "SELECT * from '" + hash_val + "'"
            df_exercisedata = pd.read_sql(sql_str, connection)
            df_exercisedata = df_exercisedata.astype(float)
            x = df_exercisedata[['x']].values
            y = df_exercisedata[['y']].values
            # hull
            hd_hull3 = calcArea(x, y, in_hd=3)
            hd_hull5 = calcArea(x, y, in_hd=5)
            hd_hull3_list.append(hd_hull3)
            hd_hull5_list.append(hd_hull5)
            # distance
            hd_distance3 = calcDistance(x, y, 3)
            hd_distance3_list.append(hd_distance3)
            hd_distance5 = calcDistance(x, y, 5)
            hd_distance5_list.append(hd_distance5)

            # error
            hd_error3 = calcError(x, y, 3)
            hd_error5 = calcError(x, y, 5)
            hd_error3_list.append(hd_error3)
            hd_error5_list.append(hd_error5)
        else:
            hd_hull3_list.append(0)
            hd_hull5_list.append(0)
            hd_distance3_list.append(0)
            hd_distance5_list.append(0)
            hd_error3_list.append(0)
            hd_error5_list.append(0)
    else:
        hd_hull3_list.append(0)
        hd_hull5_list.append(0)
        hd_distance3_list.append(0)
        hd_distance5_list.append(0)
        hd_error3_list.append(0)
        hd_error5_list.append(0)

In [399]:
df_event["H3 Distance"] = hd_distance3_list
df_event["H5 Distance"] = hd_distance5_list
df_event["H3 Error"] = hd_error3_list
df_event["H5 Error"] = hd_error5_list
df_event["H3 Hull"] = hd_hull3_list
df_event["H5 Hull"] = hd_hull5_list

sumlist = ("H3 Distance", "H5 Distance", "H3 Error", "H5 Error", "H3 Hull", "H5 Hull")

for entry in sumlist:
    print(entry)
    df_event['Sum ' + entry] = df_event['D'] + df_event['T'] + df_event['E'] + df_event[entry]
df_event

H3 Distance
H5 Distance
H3 Error
H5 Error
H3 Hull
H5 Hull


,index,Rank,Routine,Name,D,E,T,H,Pen.,Total,...,H3 Error,H5 Error,H3 Hull,H5 Hull,Sum H3 Distance,Sum H5 Distance,Sum H3 Error,Sum H5 Error,Sum H3 Hull,Sum H5 Hull
0,0,1,1st,PAGE Bryony,15.0,15.8,15.22,9.6,0.0,55.62,...,9.65,9.45,9.7,9.5,55.42,55.02,55.67,55.47,55.72,55.52
1,1,1,2nd,PAGE Bryony,14.6,15.5,14.90,8.9,-0.2,53.70,...,8.45,7.40,9.4,9.0,53.80,53.00,53.45,52.40,54.40,54.00
2,2,2,1st,SONGHURST Isabelle,13.8,16.0,15.41,9.4,0.0,54.61,...,9.25,8.75,9.7,9.5,54.61,54.21,54.46,53.96,54.91,54.71
3,3,2,2nd,SONGHURST Isabelle,3.0,3.0,3.34,1.8,0.0,11.14,...,1.65,1.40,2.0,2.0,11.34,11.34,10.99,10.74,11.34,11.34
4,4,3,1st,MAHSUDOVA Seljan,13.5,15.6,14.79,9.4,0.0,53.29,...,9.25,8.75,9.7,9.5,53.29,52.89,53.14,52.64,53.59,53.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,107,54,2nd,AUBREE Perle,9.2,13.0,12.00,7.2,0.0,41.40,...,7.15,6.60,7.7,7.5,41.60,41.20,41.35,40.80,41.90,41.70
108,108,55,1st,PENEVA Mariela,8.7,11.9,11.69,7.2,0.0,39.49,...,7.25,6.80,7.7,7.5,39.69,39.29,39.54,39.09,39.99,39.79
109,109,55,2nd,PENEVA Mariela,7.4,10.2,10.31,6.7,0.0,34.61,...,6.65,6.45,7.0,7.0,34.91,34.91,34.56,34.36,34.91,34.91
110,110,56,1st,DAVIS Augustus,2.1,3.1,3.07,1.7,0.0,9.97,...,1.55,1.25,2.0,2.0,10.27,10.27,9.82,9.52,10.27,10.27


In [459]:
rankchange_list = []
meanchange_list = []
for entry in sumlist:
    entry_full = 'Sum ' + entry
    idx_list = []
    for idx, row in df_event.iterrows():
        if idx > 0:
            if row[entry] == 0.0:
                    # print("Dropped because no file")
                    idx_list.append(idx)
            else:
                if (row["Routine"] == '2nd') and df_event.iloc[idx-1]["Routine"] == '1st':
                    if (row[entry] > df_event.iloc[idx-1][entry]):
                        idx_list.append(idx-1)
                    else:
                        idx_list.append(idx)

    df_result = df_event.drop(idx_list, axis=0)
    df_result = df_result.sort_values(by=[entry_full], ascending=False)
    new_rank = df_result["Rank"]
    df_result['New Rank'] = np.sort(new_rank)
    rank_change = np.abs(df_result['New Rank']-df_result['Rank'])
    rankchange_list.append(np.sum(rank_change!=0))
    meanchange_list.append(np.mean(rank_change))

# Full Analysis

In [495]:
header = ("N H3 Distance", "N H5 Distance", "N H3 Error", "N H5 Error", "N H3 Hull", "N H5 Hull", "MEAN H3 Distance", "MEAN H5 Distance", "MEAN H3 Error", "MEAN H5 Error", "MEAN H3 Hull", "MEAN H5 Hull")
df_main = pd.DataFrame(columns=header)
sumlist = ("H3 Distance", "H5 Distance", "H3 Error", "H5 Error", "H3 Hull", "H5 Hull")
# df_main.columns = 
df_pos = 0
gender_list = []
phase_list = []
event_list = []
event_phases = ['Qualification', 'Semi', 'Final']
event_gender = ['Women', 'Men']
df = pd.read_sql("SELECT * from ranklists", connection)
df['Event Name'] = df["Year"].astype(str) + " " + df["Event"]
events = df["Event Name"].unique().tolist()
# event_str = events[0]
for event_str in events:
    for phase in event_phases:
        for gender in event_gender:
            df_event = df[((df['Event Name']==event_str) & (df['Phase']==phase) & (df['Gender']==gender))].copy()
            df_event = df_event.reset_index()

            hd_distance3_list = []
            hd_distance5_list = []
            hd_error3_list = []
            hd_error5_list = []
            hd_hull3_list = []
            hd_hull5_list = []
            for idx, row in df_event.iterrows():
                gender = row['Gender']
                athlete = row['Name']
                athlete_name = athlete.split()
                athlete_name = athlete_name[0].capitalize() + ' ' + athlete_name[1]
                if row['Routine'] == '1st':
                    exercise = 0
                elif row['Routine'] == '2nd':
                    exercise = 1
                phase_select = df_event["Phase"].iloc[int(exercise)]
                routine_select = df_event["Routine"].iloc[int(exercise)]
                sql_str = "SELECT * from '" + event_str.replace(" ", "_").lower() + "_" + gender.lower() + "' where name=" + "'" + athlete_name + "' and phase=" + "'" + phase_select + "' and Routine=" + "'" + routine_select + "'"
                df_athlete = pd.read_sql(sql_str, connection)
                if len(df_athlete) > 0:
                    hash_val = df_athlete["Hash"].iloc[0]

                    if hash_val is not '':
                        sql_str = "SELECT * from '" + hash_val + "'"
                        df_exercisedata = pd.read_sql(sql_str, connection)
                        df_exercisedata = df_exercisedata.astype(float)
                        x = df_exercisedata[['x']].values
                        y = df_exercisedata[['y']].values
                        # hull
                        hd_hull3 = calcArea(x, y, in_hd=3)
                        hd_hull5 = calcArea(x, y, in_hd=5)
                        hd_hull3_list.append(hd_hull3)
                        hd_hull5_list.append(hd_hull5)
                        # distance
                        hd_distance3 = calcDistance(x, y, 3)
                        hd_distance3_list.append(hd_distance3)
                        hd_distance5 = calcDistance(x, y, 5)
                        hd_distance5_list.append(hd_distance5)

                        # error
                        hd_error3 = calcError(x, y, 3)
                        hd_error5 = calcError(x, y, 5)
                        hd_error3_list.append(hd_error3)
                        hd_error5_list.append(hd_error5)
                    else:
                        hd_hull3_list.append(0)
                        hd_hull5_list.append(0)
                        hd_distance3_list.append(0)
                        hd_distance5_list.append(0)
                        hd_error3_list.append(0)
                        hd_error5_list.append(0)
                else:
                    hd_hull3_list.append(0)
                    hd_hull5_list.append(0)
                    hd_distance3_list.append(0)
                    hd_distance5_list.append(0)
                    hd_error3_list.append(0)
                    hd_error5_list.append(0)

            df_event["H3 Distance"] = hd_distance3_list
            df_event["H5 Distance"] = hd_distance5_list
            df_event["H3 Error"] = hd_error3_list
            df_event["H5 Error"] = hd_error5_list
            df_event["H3 Hull"] = hd_hull3_list
            df_event["H5 Hull"] = hd_hull5_list

            

            for entry in sumlist:
                # print(entry)
                df_event['Sum ' + entry] = df_event['D'] + df_event['T'] + df_event['E'] + df_event[entry]
                # df_event
            rankchange_list = []
            meanchange_list = []
            for entry in sumlist:
                entry_full = 'Sum ' + entry
                idx_list = []
                for idx2, row2 in df_event.iterrows():
                    if idx2 > 0:
                        if row2[entry] == 0.0:
                                # print("Dropped because no file")
                                idx_list.append(idx2)
                        else:
                            if phase == "Qualification":
                                if (row2["Routine"] == '2nd') and df_event.iloc[idx2-1]["Routine"] == '1st':
                                    if (row2[entry] > df_event.iloc[idx-1][entry]):
                                        idx_list.append(idx2-1)
                                    else:
                                        idx_list.append(idx2)

                df_result = df_event.drop(idx_list, axis=0)
                df_result = df_result.sort_values(by=[entry_full], ascending=False)
                new_rank = df_result["Rank"]
                df_result['New Rank'] = np.sort(new_rank)
                rank_change = np.abs(df_result['New Rank']-df_result['Rank'])
                rankchange_list.append(np.sum(rank_change!=0))
                meanchange_list.append(np.mean(rank_change))
        
            df_main.loc[df_pos] = rankchange_list + meanchange_list
            gender_list.append(gender)
            phase_list.append(phase)
            event_list.append(event_str)
            df_pos += 1

In [496]:
df_main['Event'] = df['Event Name']
df_main['Gender'] = gender_list
df_main['Phase'] = phase_list
df_main['Event'] = event_list

In [497]:
df_main.to_csv('rank_change_all.csv')

In [494]:
df_main

,N H3 Distance,N H5 Distance,N H3 Error,N H5 Error,N H3 Hull,N H5 Hull,MEAN H3 Distance,N H5 Distance,N H3 Error,N H5 Error,N H3 Hull,N H5 Hull,Event,Gender,Phase
0,50.0,51.0,49.0,51.0,51.0,52.0,8.792453,8.905660,10.150943,10.452830,8.679245,8.716981,2022 European Championships,Women,Qualification
1,53.0,54.0,56.0,57.0,54.0,53.0,10.526316,10.526316,11.793103,11.793103,10.350877,10.245614,2022 European Championships,Men,Qualification
2,2.0,2.0,6.0,8.0,4.0,4.0,0.095238,0.095238,0.285714,0.380952,0.190476,0.190476,2022 European Championships,Women,Semi
3,13.0,14.0,13.0,13.0,12.0,9.0,0.761905,0.857143,0.761905,1.047619,0.761905,0.571429,2022 European Championships,Men,Semi
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2022 European Championships,Women,Final
5,0.0,2.0,2.0,2.0,2.0,2.0,0.000000,0.250000,0.250000,0.250000,0.250000,0.250000,2022 European Championships,Men,Final
6,46.0,46.0,46.0,47.0,43.0,46.0,2.980392,3.176471,3.098039,3.137255,2.941176,3.058824,2022 European Championships,Women,Qualification
7,70.0,69.0,72.0,72.0,69.0,66.0,6.921053,7.315789,7.000000,7.500000,6.473684,6.605263,2022 European Championships,Men,Qualification
8,5.0,6.0,4.0,10.0,8.0,8.0,0.260870,0.347826,0.347826,0.695652,0.521739,0.521739,2022 European Championships,Women,Semi
9,2.0,8.0,8.0,11.0,11.0,13.0,0.083333,0.500000,0.333333,0.583333,0.500000,0.583333,2022 European Championships,Men,Semi


In [491]:
events

['2022 European Championships',
 '2021 World Championships',
 '2019 World Championships',
 '2020 Olympic Games']